# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup and Imports

In [1]:
# For example: 
import os
from dotenv import load_dotenv                  # For loading environment variables from .env file
from datetime import datetime                   # For websearch metadata assignment

from typing import List, Dict
from pydantic import BaseModel, Field

from lib.agents import Agent                    # For Agent class
from lib.llm import LLM                         # For Large Language Model
from lib.messages import UserMessage, SystemMessage, BaseMessage
from lib.tooling import tool                    # For defining tools    
from lib.parsers import PydanticOutputParser    # To return structured output from LLM

from chromadb import PersistentClient           # For Vector Database
# import chromadb

from tavily import TavilyClient                 # For Web Search


In [2]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
VOCAREUM_OPENAI_API_KEY=os.getenv("VOCAREUM_OPENAI_API_KEY")
VOCAREUM_BASE_URL=os.getenv("VOCAREUM_BASE_URL")

Setting up ChromaDB paths and names. Should be consistent with what you used in the `Udaplay_01.ipynb` file.

In [3]:
CHROMADB_PATH = "chromadb"
CHROMADB_COLLECTION_NAME = "udaplay"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
@tool
def retrieve_game(query: str, n_results: int=4) -> List:
    """
#    Semantic search: Finds most relevant results in the vector DB
#    args:
#    - query: a question about game industry. 
#    - n_results: number of results to return (default is 4)
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
    """

    # collection.query(
        # query_texts=[query],
        # n_results=n_results,
        # include=['metadatas', 'documents', 'distances'] # TODO: look at collection.peek() to understand this better.
    # )

    # These have been created in Phase 1
    chroma_client = PersistentClient(path=CHROMADB_PATH)
    collection = chroma_client.get_collection(CHROMADB_COLLECTION_NAME)

    results = collection.query(
        query_texts=[query], 
        n_results=5
        )
    
    return results["documents"][0]

In [5]:
# To test the function
# query = "Tell me about Gran Turismo"
# response = retrieve_game(query)
# print(response)

#### Evaluate Retrieval Tool

In [6]:
class EvaluationReport(BaseModel):
    useful: bool = Field(..., description="Indicates if the retrieved documents are useful for answering the question")
    description: str = Field(..., description="Detailed explanation of the evaluation result, including why the documents are or are not useful")

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents, this function 
    will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    Outputs:
    EvaluationReport:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    system_prompt = """
        You are an expert in evaluating the usability of documents for answering user queries.
        Your task is to evaluate if the provided sources are enough to respond the user query.
        Give a detailed explanation, so it's possible to take an action to accept it or not.
    """

    user_prompt = f"""
        User query:\n{question}\n
        Retrieved documents:\n{retrieved_docs}
    """

    messages = [
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    llm_judge = LLM(
        model="gpt-4.1-nano",
        temperature=0.0,
        api_key=VOCAREUM_OPENAI_API_KEY,
        base_url=VOCAREUM_BASE_URL
    )

    ai_message = llm_judge.invoke(input=messages, response_format=EvaluationReport)
    parser = PydanticOutputParser(model_class=EvaluationReport)
    return parser.parse(ai_message)


In [7]:
# To test the function
# response_valid = evaluate_retrieval(query, response)
# print(response_valid.model_dump_json(indent=2))

#### Game Web Search Tool

In [8]:
class MetaData(BaseModel):
    timestamp: str = Field(..., description="The time the web search was performed, in ISO format (YYYY-MM-DD HH:MM:SS.mmmmmm)")
    query: str = Field(..., description="The original search query")

class WebSearchResult(BaseModel):
    answer: str = Field(..., description="Answer to the search query")
    results: List[Dict] = Field(..., description="List of search results")
    search_metadata: MetaData  = Field(..., description="Metadata of the web search")

# Modified from Course module 06 solution notebook (Web Search with Tavily)
@tool
def game_web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API.
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    Output: 
    WebSearchResult:
    - answer (str): Direct answer to the query, if available
    - results (List[Dict]): List of search results, each containing:
    - search_metadata (Dict): Metadata about the search, including timestamp and query
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    # metadata_parser = PydanticOutputParser(model_class=WebSearchResult)
    # validated_results = metadata_parser.parse(ai_message=AIMessage(content=str(formatted_results)))
    validated_results = WebSearchResult(**formatted_results)

    return validated_results

In [9]:
# To test the function
# web_search_response = game_web_search(query)
# print(web_search_response.model_dump_json(indent=2))

### Creating and invoking the Agent

In [10]:
tools = [retrieve_game, evaluate_retrieval, game_web_search]

instructions = f"""
    You are a helpful assistant that answers questions about video games. 
    Users will ask you about video games, including names, contents, publishing years, etc.

    You should first check whether an answer to the question is possible based on your knowledge (stored as a VectorDB), using the `retrieve_game` tool.
    After using `retrieve_game`, always run `evaluate_retrieval` to check whether the retrieved information is actually useful. 
    If the result of `evaluate_retrieval` is that the information is not useful, use your `game_web_search` tool to find an answer on the web. 

    Final output: 
    - In your final answer, always state whether the answer was from the curated database or from the open web.
    - If you found an answer on the web, cite the highest-ranked URL you found, so that users can go look up the primary source. The users don't have access to the search results, so cite a URL with link. Always include a link in your answer.
    - Do not make up facts.

    EXAMPLEs for final answer: 
    - Mario Kart is a racing game developed for the Nintendo GameCube. This information was retrieved from the internal database. 
    - Zelda: Tears of the Kingdom was released in 2023. This information was found on this website: https://www.nintendo.com/en-gb/Games/Nintendo-Switch-games/The-Legend-of-Zelda-Tears-of-the-Kingdom-1576884.html. 
"""

web_agent = Agent(
    model_name="gpt-4.1-nano",
    instructions=instructions,
    tools=tools,
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,
)

In [11]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

In [12]:
def print_agent_answer(messages: List[BaseMessage]):
    # Get tool calls invoked
    tools_called = []
    index_last_query = next(i for i, m in reversed(list(enumerate(messages))) if m.role == 'user')
    # print(f"index: {index_last_query} out of {len(messages)}") # To ensure the previous line works as expected
    for m in messages[index_last_query:]: 
        if getattr(m, 'tool_calls', None):
            tools_called.append(m.tool_calls[0].function.name)
    print(f"Final answer from the agent: {messages[-1].content}")
    print(f"The agent called these tools (in order): {tools_called}")

In [13]:
query1="When were Pokémon Gold and Silver released?"
sessionid1="pokemon"

run1 = web_agent.invoke(
    query=query1,
    session_id=sessionid1
)

messages = run1.get_final_state()["messages"]
# print("\nMessages from run 1:")
# print_messages(messages)              # To print much more information on intermediate steps
print("-" * 30)
print(f"Query: {query1}")
print_agent_answer(messages)
print("-" * 50)

query1_followup="Thanks, and which version was released in 2002?"

run1_followup = web_agent.invoke(
    query=query1_followup,
    session_id=sessionid1
)

messages_followup = run1_followup.get_final_state()["messages"]
# print("\nMessages from run 1 (followup):")
# print_messages(messages_followup)         # To print much more information
print("-" * 30)
print(f"Query: {query1_followup}")
print_agent_answer(messages_followup)
print("-" * 50)



[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
------------------------------
Query: When were Pokémon Gold and Silver released?
Final answer from the agent: Pokémon Gold and Silver were released in 1999. This information was retrieved from the internal database.
The agent called these tools (in order): ['retrieve_game', 'evaluate_retrieval']
--------------------------------------------------
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor

In [14]:
# For debugging and improvements: 
# print(messages)
# print_messages(messages=messages)

In [15]:
query2="Which one was the first 3D platformer Mario game?"
sessionid2="mario"
run2 = web_agent.invoke(
    query=query2,
    session_id=sessionid2,
)

messages = run2.get_final_state()["messages"]
# print("\nMessages from run 2:")
# print_messages(messages)  # To print much more information
print("-" * 30)
print(f"Query: {query2}")
print_agent_answer(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
------------------------------
Query: Which one was the first 3D platformer Mario game?
Final answer from the agent: The first 3D platformer Mario game was "Super Mario 64," released in 1996 for the Nintendo 64. This information was retrieved from the internal database.
The agent called these tools (in order): ['retrieve_game', 'evaluate_retrieval']


In [16]:
query3="Was Mortal Kombat X realeased for Playstation 5?"
sessionid3="mortal"

run3 = web_agent.invoke(
    query=query3,
    session_id=sessionid3,
)

messages = run3.get_final_state()["messages"]
# print("\nMessages from run 3:")
# print_messages(messages)          # To print much more information

print("-" * 30)
print(f"Query: {query3}")
print_agent_answer(messages)


[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
------------------------------
Query: Was Mortal Kombat X realeased for Playstation 5?
Final answer from the agent: Mortal Kombat X was originally released on April 14, 2015, for PlayStation 4. It is playable on PlayStation 5 through backward compatibility, but some features may be absent. The game can be updated to the latest system software on PS5 for better compatibility. This information was found on the PlayStation Store and other sources. 

You can see more details here: [PlayStation Store - Mortal Kombat X](https://store.playstation.com/en-us/produc

### (Optional) Advanced

In [17]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes